In [16]:
! echo $PATH
! type python
! type python3
! type pip
! type pip3

/Library/Frameworks/Python.framework/Versions/3.7/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Library/TeX/texbin:/usr/local/share/dotnet:~/.dotnet/tools
python is /usr/bin/python
python3 is /Library/Frameworks/Python.framework/Versions/3.7/bin/python3
pip is /usr/local/bin/pip
pip3 is /usr/local/bin/pip3


<img src='task_description.jpg'>

In [188]:
! python3 -m pip install tqdm

     |████████████████████████████████| 66 kB 492 kB/s eta 0:00:011     |███████████████████▋            | 40 kB 527 kB/s eta 0:00:01


In [53]:
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from numpy.random import seed
seed(1337)
from tensorflow.random import set_seed
set_seed(42)

from tensorflow.python.keras.applications import vgg16
from tensorflow.python.keras.applications.vgg16 import preprocess_input
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.python.keras.callbacks import ModelCheckpoint
from tensorflow.python.keras import layers, models, Model, optimizers

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from PIL import Image

In [54]:
def get_img(df, idx):
    row = df.iloc[idx]
    filename = row.url.split('/')[-1]
    print(filename)
    path = os.path.join(str(row.id), row.color, row.source, filename)
    print(path)
    return Image.open(path)

In [56]:
df = pd.read_csv('clothes-dataset.csv')

In [57]:
df.head()

,id,category,color,source,url
0,32712227764,Hoodies & Sweatshirts,1,customer,https://ae01.alicdn.com/kf/UTB8wgKZASnEXKJk43U...
1,32712227764,Hoodies & Sweatshirts,1,customer,https://ae01.alicdn.com/kf/UTB8xiXPgpPJXKJkSah...
2,32712227764,Hoodies & Sweatshirts,1,customer,https://ae01.alicdn.com/kf/UTB8_rEgCVfFXKJk43O...
3,32712227764,Hoodies & Sweatshirts,1,customer,https://ae01.alicdn.com/kf/UTB82d1_ggnJXKJkSae...
4,32712227764,Hoodies & Sweatshirts,1,customer,https://ae01.alicdn.com/kf/UTB8xhnbgmbIXKJkSae...


In [113]:
df.groupby('category')['url'].nunique().where(lambda x: x > 1148).dropna().sort_values(ascending=False)

category
T-Shirts                 197323.0
Hoodies & Sweatshirts     72192.0
Ankle Boots               19499.0
Skirts                    15867.0
Pants & Capris            12840.0
Blouses & Shirts          12565.0
Dresses                    6394.0
Jeans                      6127.0
Tank Tops                  4188.0
Leggings                   3740.0
Casual Shirts              3424.0
Polo                       2809.0
Shorts                     2211.0
Pullovers                  2106.0
Sweatpants                 1920.0
Jumpsuits                  1549.0
Casual Pants               1291.0
Name: url, dtype: float64

In [115]:
relevant_categories = np.array(df.groupby('category')['url'].nunique().where(lambda x: x > 1148).dropna().sort_values(ascending=False).index)
print(relevant_categories)

['T-Shirts' 'Hoodies & Sweatshirts' 'Ankle Boots' 'Skirts'
 'Pants & Capris' 'Blouses & Shirts' 'Dresses' 'Jeans' 'Tank Tops'
 'Leggings' 'Casual Shirts' 'Polo' 'Shorts' 'Pullovers' 'Sweatpants'
 'Jumpsuits' 'Casual Pants']


In [116]:
df2 = df[df['category'].isin(relevant_categories)]
# оставим только категории, где больше 1148 фото (начиная от блока два шага назад)

In [117]:
df2.shape

(366749, 5)

In [165]:
df_category = [pd.DataFrame() for i in range(len(relevant_categories))]
for i, category in enumerate(relevant_categories):
    df_category[i] = df2[df2['category'] == category]
    if len(df2[df2['category'] == category]) > 10000:
        df_category[i] = pd.DataFrame(df_category[i].groupby('id')['url'].first().reset_index(drop=True))
        df_category[i]['category'] = category
    else:
        df_category[i] = df_category[i].drop(columns=['id', 'source', 'color']).reset_index(drop=True)
    print(category + ' ' + str(len(df_category[i])))
    

T-Shirts 14335
Hoodies & Sweatshirts 5529
Ankle Boots 1417
Skirts 1138
Pants & Capris 847
Blouses & Shirts 1157
Dresses 6410
Jeans 6128
Tank Tops 4196
Leggings 3764
Casual Shirts 3438
Polo 2814
Shorts 2213
Pullovers 2109
Sweatpants 1927
Jumpsuits 1549
Casual Pants 1292


In [169]:
df3 = pd.concat(df_category, sort=False)
df3.reset_index(inplace=True, drop=True)
print(df3)

                                                     url      category
0      https://ae01.alicdn.com/kf/HTB1CP3qKpXXXXaXXXX...      T-Shirts
1      https://ae01.alicdn.com/kf/HTB18If9HVXXXXagXXX...      T-Shirts
2      https://ae01.alicdn.com/kf/HTB13U3ZMXXXXXXwXXX...      T-Shirts
3      https://ae01.alicdn.com/kf/HTB1p3MGu1uSBuNjSsz...      T-Shirts
4      https://ae01.alicdn.com/kf/HTB1rooRwf5TBuNjSsp...      T-Shirts
...                                                  ...           ...
60258  https://ae01.alicdn.com/kf/UTB8K7BnCpfFXKJk43O...  Casual Pants
60259  https://ae01.alicdn.com/kf/UTB8jDrslf2JXKJkSan...  Casual Pants
60260  https://ae01.alicdn.com/kf/UTB830TxlXfJXKJkSam...  Casual Pants
60261  https://ae01.alicdn.com/kf/UTB8otPxlXfJXKJkSam...  Casual Pants
60262  https://ae01.alicdn.com/kf/UTB8LzbeEiaMiuJk43P...  Casual Pants

[60263 rows x 2 columns]


In [171]:
import requests

In [189]:
from tqdm import tqdm_notebook

In [193]:
# for category in relevant_categories:
#     os.makedirs(f'photos/{category}', exist_ok=True)
# for i, row in tqdm_notebook(df3.iterrows(), total=df3.shape[0]):
#     path = f'photos/{row.category}/{i}.jpg'
#     if os.path.exists(path):
#         continue
#     try:
#         res = requests.get(row.url)
#     except:
#         print(f'Не могу скачать {row}')
#         continue
#     with open(path, 'wb') as f:
#         f.write(res.content)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


Не могу скачать url         https://ae01.alicdn.com/kf/HTB1tQFUHb1YBuNjSsz...
category                                Hoodies & Sweatshirts
Name: 18071, dtype: object
Не могу скачать url         https://ae01.alicdn.com/kf/HTB1yxgNh3ZC2uNjSZF...
category                                Hoodies & Sweatshirts
Name: 18233, dtype: object
Не могу скачать url         https://ae01.alicdn.com/kf/HTB1ihL4JY1YBuNjSsz...
category                                Hoodies & Sweatshirts
Name: 18234, dtype: object
Не могу скачать url         https://ae01.alicdn.com/kf/HTB1S_DjmBjTBKNjSZF...
category                                Hoodies & Sweatshirts
Name: 18235, dtype: object
Не могу скачать url         https://ae01.alicdn.com/kf/HTB1XSUsXffsK1RjSsz...
category                                Hoodies & Sweatshirts
Name: 19513, dtype: object
Не могу скачать url         https://ae01.alicdn.com/kf/HTB1pz8GXyLxK1Rjy0F...
category                                Hoodies & Sweatshirts
Name: 19514, dtype: objec

Не могу скачать url         https://ae01.alicdn.com/kf/HTB15zrCmC_I8KJjy0F...
category                                              Dresses
Name: 24913, dtype: object
Не могу скачать url         https://ae01.alicdn.com/kf/HTB1.xojmvDH8KJjy1X...
category                                              Dresses
Name: 24915, dtype: object
Не могу скачать url         https://ae01.alicdn.com/kf/UTB8Ntt7FarFXKJk43O...
category                                              Dresses
Name: 25075, dtype: object
Не могу скачать url         https://ae01.alicdn.com/kf/HTB1QXpFbK7EWeJjSZF...
category                                              Dresses
Name: 25078, dtype: object
Не могу скачать url         https://ae01.alicdn.com/kf/UTB8TTixFVfFXKJk43O...
category                                                Jeans
Name: 34146, dtype: object
Не могу скачать url         https://ae01.alicdn.com/kf/HTB19lkslN3IL1JjSZP...
category                                                Jeans
Name: 36268, dtype: objec

In [183]:
import io
x = Image.open(io.BytesIO(res.content))
x.show()

In [194]:
import shutil
import random

In [202]:
root_dir = 'data'

val_ratio = 0.1
test_ratio = 0.1

for category in relevant_categories:
#     os.makedirs(root_dir +'/train/' + category)
#     os.makedirs(root_dir +'/val/' + category)
#     os.makedirs(root_dir +'/test/' + category)
    
    copy_from_dir = 'photos/' + category
    
    all_filenames = os.listdir(copy_from_dir)
    np.random.shuffle(all_filenames)
    train_filenames, val_filenames, test_filenames = np.split(
        np.array(all_filenames),
        [int(len(all_filenames)* (1 - val_ratio - test_ratio)), int(len(all_filenames)* (1 - test_ratio))]
    )
    train_filenames = [copy_from_dir + '/' + name for name in train_filenames.tolist()]
    val_filenames = [copy_from_dir + '/' + name for name in val_filenames.tolist()]
    test_filenames = [copy_from_dir + '/' + name for name in test_filenames.tolist()]
    
    print('Number of total images: ', len(all_filenames))
    print('Training: ', len(train_filenames))
    print('Validation: ', len(val_filenames))
    print('Testing: ', len(test_filenames))
    
    for name in train_filenames:
        shutil.copy(name, root_dir + '/train/' + category)

    for name in val_filenames:
        shutil.copy(name, root_dir +'/val/' + category)

    for name in test_filenames:
        shutil.copy(name, root_dir +'/test/' + category)

Number of total images:  14336
Training:  11468
Validation:  1434
Testing:  1434
Number of total images:  5522
Training:  4417
Validation:  552
Testing:  553
Number of total images:  1411
Training:  1128
Validation:  141
Testing:  142
Number of total images:  1138
Training:  910
Validation:  114
Testing:  114
Number of total images:  841
Training:  672
Validation:  84
Testing:  85
Number of total images:  1140
Training:  912
Validation:  114
Testing:  114
Number of total images:  5733
Training:  4586
Validation:  573
Testing:  574
Number of total images:  6115
Training:  4892
Validation:  611
Testing:  612
Number of total images:  4195
Training:  3356
Validation:  419
Testing:  420
Number of total images:  3764
Training:  3011
Validation:  376
Testing:  377
Number of total images:  2314
Training:  1851
Validation:  231
Testing:  232
Number of total images:  2814
Training:  2251
Validation:  281
Testing:  282
Number of total images:  2213
Training:  1770
Validation:  221
Testing:  222
N

In [205]:
for category in relevant_categories:
    all_filenames = os.listdir('photos/' + category)
    for s in all_filenames:
        if s[-4:] != '.jpg':
            print(s)

.DS_Store
.DS_Store
.DS_Store
.DS_Store
.DS_Store


In [209]:
for category in relevant_categories:
    all_filenames = os.listdir('photos/' + category)
    for s in all_filenames:
        if s[-4:] != '.jpg':
            print(s)
#             print('photos/' + category + '/' + s)
#             os.remove('photos/' + category + '/' + s)